In [1]:
import numpy as np
import open3d as o3d
# import pptk # works with Python 3.6

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Load ply file
pc_ply = o3d.io.read_point_cloud("./data/P001 2022-01-25 01_39_54.ply")
print('PLY file loaded')

PLY file loaded


In [3]:
# PLY file info
print(dir(pc_ply))

['HalfEdgeTriangleMesh', 'Image', 'LineSet', 'PointCloud', 'RGBDImage', 'TetraMesh', 'TriangleMesh', 'Type', 'Unspecified', 'VoxelGrid', '__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'cluster_dbscan', 'colors', 'compute_convex_hull', 'compute_mahalanobis_distance', 'compute_mean_and_covariance', 'compute_nearest_neighbor_distance', 'compute_point_cloud_distance', 'covariances', 'create_from_depth_image', 'create_from_rgbd_image', 'crop', 'detect_planar_patches', 'dimension', 'estimate_covariances', 'estimate_normals', 'estimate_point_covariances', 'farthest_point_down_sample', 'get_axis_aligned_bounding_box', 'get_center', 'get_geometry_type', 'get_max_bound', 

In [4]:
print('Shape of points', np.asarray(pc_ply.points).shape)
print('Shape of colors', np.asarray(pc_ply.colors).shape)

Shape of points (247847, 3)
Shape of colors (247847, 3)


In [5]:
# visualise
o3d.visualization.draw_geometries([pc_ply], window_name="Original Point Cloud")

In [6]:
# downsampling
voxelSize = 0.01
downsampled = pc_ply.voxel_down_sample(voxelSize)

In [7]:
# outliers
cl, ind = downsampled.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
cleaned_pcd = downsampled.select_by_index(ind)

In [35]:

# Define the bounding box
min_x = np.min(cleaned_pcd[:, 0])
max_x = np.max(cleaned_pcd[:, 0])
min_y = np.min(cleaned_pcd[:, 1])
max_y = np.max(cleaned_pcd[:, 1])
min_z = np.min(cleaned_pcd[:, 2])
max_z = np.max(cleaned_pcd[:, 2])
print(min_x)

# Expand the bounding box slightly
padding = 0.5  # for example, 10% of the range

min_x = padding * min_x
max_x = padding * max_x
min_y = padding * min_y
max_y = padding * max_y
min_z = padding * min_z
#z neutral as don't want to cut off head
max_z = 1 * max_z

print(min_x)

# Filter the points
human_points = cleaned_pcd[
    (cleaned_pcd[:, 0] >= min_x) & (cleaned_pcd[:, 0] <= max_x) &
    (cleaned_pcd[:, 1] >= min_y) & (cleaned_pcd[:, 1] <= max_y) &
    (cleaned_pcd[:, 2] >= min_z) & (cleaned_pcd[:, 2] <= max_z)
]


###
visualized_point_cloud = o3d.geometry.PointCloud()
visualized_point_cloud.points = o3d.utility.Vector3dVector(human_points)
o3d.visualization.draw_geometries([visualized_point_cloud])

print("done")

TypeError: 'open3d.cpu.pybind.geometry.PointCloud' object is not subscriptable

In [31]:
plane_model, inliers = cleaned_pcd.segment_plane(distance_threshold=0.009, ransac_n=3, num_iterations=10000)
inlier_cloud = cleaned_pcd.select_by_index(inliers)
outlier_cloud = cleaned_pcd.select_by_index(inliers, invert=True)

# Visualize the cleaned point cloud
o3d.visualization.draw_geometries([outlier_cloud], window_name="Cleaned Point Cloud")


In [15]:
o3d.visualization.draw_geometries([outlier_cloud], window_name="Cleaned Point Cloud")